In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
%config Completer.use_jedi = True
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import scalp
from scalp.output import draw
from scalp import data,  test_config

In [ ]:
if True:
    import better_exceptions
    import sys
    ip = get_ipython()
    def custom_exc(shell, etype, evalue, tb, tb_offset=None):
        better_exceptions.excepthook(etype, evalue, tb)
    ip.set_custom_exc((Exception,), custom_exc)

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", module="anndata")

In [ ]:
# datasets_ts = data.loaddata_timeseries(test_config.timeseries_datapath,  maxdatasets=5,  maxcells = 600,  datasets = ['s5'])
datasets_ts = data.loaddata_timeseries(test_config.timeseries_datapath,  maxdatasets=5,  maxcells = 1000)
dataset_ts = datasets_ts[0]

In [ ]:
# datasets = data.loaddata_scib(test_config.scib_datapath,  maxdatasets=5,  maxcells = 1000,  datasets = ["Immune_ALL_hum_mou"])
datasets = data.loaddata_scib(test_config.scib_datapath,  maxdatasets=3,  maxcells = 400 )

dataset = datasets[0]






In [ ]:
from scalp.pca import pca
dataset = pca(dataset, dim=2, label ='pca2')

In [ ]:
from scalp.output import draw 
draw.snsplot(dataset)

In [ ]:
for dataset in datasets:
    # joined UMAP
    dataset = scalp.umapwrap.adatas_umap(dataset,dim = 2 ,from_obsm=False ,label='u2')
    scalp.snsplot(dataset,coordinate_label = 'u2', splitby = 'label') 
    draw.batchplot(dataset, from_obsm = 'u2')

In [ ]:
import scalp
import scalp.output.draw as draw
from sklearn.manifold import TSNE
for dataset in datasets:
    # joined UMAP
    dataset = scalp.umapwrap.adatas_umap(dataset,dim = 2 ,from_obsm=False ,label='u2')
    scalp.snsplot(dataset,coordinate_label = 'u2', splitby = 'batch')
    draw.batchplot(dataset, from_obsm = 'u2')
    #  independant TSNE 
    for data in dataset:
        data.obsm['TSNE'] = TSNE(perplexity=15).fit_transform(data.X.todense().A )
    scalp.snsplot(dataset,coordinate_label = 'TSNE')
    


    # also add our method
    dataset, grap = scalp.mkgraph(dataset,intra_neigh = 7, 
                                  inter_neigh=1,add_tree=False)
    scalp.graph_embed_plot(dataset, grap)
    draw.batchplot(dataset, from_obsm = 'embedding')


In [ ]:
for x in [3,4,5]:
    dataset, grap = scalp.mkgraph(dataset,neighbors_total = 30,  neighbors_intra_fraction=x/10,add_tree=False)
    dataset = scalp.umapwrap.graph_umap(dataset,grap,label = 'embedding')
    scalp.umapwrap.graph_jax(dataset,grap,label = 'umap', n_components = 2, neighbors_total = 30)
    # draw.batchplot(dataset, from_obsm = 'embedding')
    draw.batchplot(dataset, from_obsm = 'umap')

In [ ]:
dataset, graph = scalp.mkgraph(dataset,neighbors_total = 30, 
                              intra_neighbors_mutual=False,
                              neighbors_intra_fraction=.3,add_tree=False)
# dataset = scalp.umapwrap.graph_umap(dataset,graph,label = 'embedding')
scalp.umapwrap.graph_umap(dataset,graph,label = 'umap', n_components = 2)
#scalp.umapwrap.graph_jax(dataset,grap,label = 'umap', n_components = 2, neighbors_total = 20)
draw.batchplot(dataset, from_obsm = 'umap')

In [ ]:
dataset, grap = scalp.mkgraph(dataset,intra_neigh = 10, scaling_threshold = .99, 
                              inter_neigh=5,add_tree=False)
dataset = scalp.umapwrap.graph_umap(dataset,grap,label = 'embedding')
draw.batchplot(dataset, from_obsm = 'embedding')

In [ ]:
grap

In [ ]:
dataset, grap = scalp.mkgraph(dataset,neighbors_total = 30, 
                              neighbors_intra_fraction = .5,
                              add_tree=False)

#    dataset = scalp.mnn.scanorama(dataset,label = 'scano', base='pca')
#    dataset = scalp.umapwrap.adatas_umap(dataset,from_obsm = 'scano', label='scanoUmap', dim = 2, n_neighbors = 40)
#    draw.batchplot(dataset, from_obsm = 'scanoUmap')

dataset = scalp.umapwrap.graph_umap(dataset,grap,label = 'embedding')
draw.batchplot(dataset, from_obsm = 'embedding')
    
# dataset = scalp.umapwrap.graph_NX(dataset,grap,algo = 'kamada_kawai',label = 'kawaii')
# dataset = scalp.umapwrap.graph_NX(dataset,grap,algo = 'spring', label = 'spring')
# draw.batchplot(dataset, from_obsm = 'spring')
# draw.batchplot(dataset, from_obsm = 'kawaii')

# dataset_ts, grap = scalp.mkgraph(dataset_ts,intra_neigh = ol, scaling_threshold = 1, outlier_threshold = .7,
# inter_neigh=1,add_tree=False)
# dataset_ts = scalp.umapwrap.graph_umap(dataset_ts,grap,label = 'embedding')
# draw.batchplot(dataset_ts, from_obsm = 'embedding')

#  linecopy False, intra6 and inter3 look good btw
plt.matshow(grap.todense())

In [ ]:
import scipy.sparse as sparse
a = sparse.csr_matrix([0,0,1,5])

In [ ]:
print(a.astype(bool)*4)

In [ ]:
a.data = np.full_like(a.data, 4)

In [ ]:
import numpy as np
a = np.zeros((4, 4))
np.fill_diagonal(a, 1)
a[0,3]=1
a[3,0]=1
a[2,0]=1
a[1,2]=1
a[2,3]=1
print(a)
print(a*a.T)

In [ ]:
for x in [2]:
    dataset, grap = scalp.mkgraph(dataset,intra_neigh = 16, 
                                  inter_neigh=x,
                                  add_tree=True,
                                  outlier_threshold=1)
    dataset = scalp.umapwrap.graph_umap(dataset,grap,label = 'embedding')
    draw.batchplot(dataset, from_obsm = 'embedding')




In [ ]:
%%time 
from scalp.data.similarity import make_stairs
dataset_ts, grapt = scalp.mkgraph(dataset_ts,
                                  intra_neigh = 3, 
                                  intra_neighbors_mutual = True,
                                  inter_neigh=1,
                                  add_tree=True, 
                                  copy_lsa_neighbors = False,
                                  inter_outlier_threshold = None,
                                  inter_outlier_probabilistic_removal =True,
                                  dataset_adjacency = make_stairs(len(dataset_ts),[0,1,2]) )
dataset_ts = scalp.umapwrap.graph_umap(dataset_ts,grapt,label = 'embedding')
draw.batchplot(dataset_ts, from_obsm = 'embedding')

In [ ]:
%%time 
from scalp.data.similarity import make_stairs
dataset, grap = scalp.mkgraph(dataset,
                                  intra_neigh = 16, 
                                  intra_neighbors_mutual = True,
                                  inter_neigh=5,
                                  add_tree=True, 
                                  copy_lsa_neighbors = True,
                                  inter_outlier_threshold = None,
                                  inter_outlier_probabilistic_removal =True)
dataset = scalp.umapwrap.graph_umap(dataset,grap,label = 'embedding')
draw.batchplot(dataset, from_obsm = 'embedding')

In [ ]:
plt.matshow(grapt.todense()[:600,:600])

In [ ]:
bestparms = {'intra_neigh': 4, 'intra_neighbors_mutual': True, 'inter_neigh': 3, 'add_tree': True, 'copy_lsa_neighbors': False,
             'inter_outlier_threshold': 0.75, 'inter_outlier_probabilistic_removal': True}
for ds in datasets_ts:
    ds, grapt = scalp.mkgraph(ds, intra_neigh = 7,  inter_neigh=1, add_tree=True,  outlier_threshold = 1, dataset_adjacency = make_stairs(len(ds),[0,1]) )
    # ds, grapt = scalp.mkgraph(ds, **bestparms)
    ds = scalp.umapwrap.graph_umap(ds,grapt,label = 'embedding')
    draw.batchplot(ds, from_obsm = 'embedding')

In [ ]:
bestparams = {'intra_neigh': 4, 'intra_neighbors_mutual': False, 'inter_neigh': 4, 'add_tree': True, 'copy_lsa_neighbors': False, 'inter_outlier_threshold': 0.6, 'inter_outlier_probabilistic_removal': False}
best_silh = {'intra_neigh': 6, 'intra_neighbors_mutual': False, 'inter_neigh': 4, 'add_tree': True,  'copy_lsa_neighbors': False, 'inter_outlier_threshold': 0.6, 'inter_outlier_probabilistic_removal': False}
for ds in datasets:
    # ds, grapt = scalp.mkgraph(ds, intra_neigh = 7,  inter_neigh=1, add_tree=True,  outlier_threshold = 1, dataset_adjacency = make_stairs(len(ds),[0,1]) )
    ds, grapt = scalp.mkgraph(ds, **notperfectparams)
    ds = scalp.umapwrap.graph_umap(ds,grapt,label = 'embedding')
    draw.batchplot(ds, from_obsm = 'embedding')

In [ ]:
notperfectparams = {'intra_neigh': 8, 'intra_neighbors_mutual': False, 'inter_neigh': 4, 'add_tree': True,
              'copy_lsa_neighbors': False, 'inter_outlier_threshold': 0.95, 'inter_outlier_probabilistic_removal': False}
ds =  datasets[2]
ds, grapt = scalp.mkgraph(ds, **bestparams)
ds = scalp.umapwrap.graph_umap(ds,grapt,label = 'embedding')
draw.batchplot(ds, from_obsm = 'embedding')

In [ ]:
from scipy import sparse
z =sparse.lil_matrix([[1,2,3,0],[],[0,0,0,1]])
print (z.__dict__)
for zz in z:
    print (zz.rows[0])
    print()

In [ ]:
import numpy as np
zz = np.random.rand(4)
zz

In [ ]:
zz > .3